[질문]
1. "text.Tokenizer()" 토큰화라는 명칭을 함수에서 임베딩을 하고 있음 --> 책에서 배운 "토큰화, 임베딩" 개념하고 달라 혼란스러움
- 책에서 말하는 토큰화를 여기서는 "preprocessing.text.text_to_word_sequence"로 음절 분리를 하고 있음
- 책에서 말하는 임베딩을 여기서는 "text.Tokenizer()"를 통해 라벨 인코딩을 하고 있음



In [4]:
# 6-3

# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate


# 데이터 읽어오기
train_file = "../../../chatbot_book_ex/book_ex/ch6/chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()


# 1. 음절단위 분류  2. 토크나이저 핏 3. 토크나이저에서 시퀀스 추출  4. 3을 패딩

# 단어 인덱스 시퀀스 벡터, 음절단위 분류
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
print("corpus[0:10]", corpus[0:10])


# 단어 토큰화??, 음절단위 분리 후 또하는 이유는?
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)


# 임베딩, 라벨 인코더
sequences = tokenizer.texts_to_sequences(corpus)  # 총 단어에 각기 다른 숫자를 부여한 리스트 반환 --> 임베딩의 한 방법중 라벨 인코딩 같음
word_index = tokenizer.word_index
print(type(sequences))
print("sequences[0:10]", sequences[0:10])


# 15개 단어 차원으로 패딩이 들어감
MAX_SEQ_LEN = 15  # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')
print(type(padded_seqs))
print(padded_seqs[0:10])


# # 학습용, 검증용, 테스트용 데이터셋 생성 ➌
# # 학습셋:검증셋:테스트셋 = 7:2:1
# ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
# ds = ds.shuffle(len(features))

# train_size = int(len(padded_seqs) * 0.7)
# val_size = int(len(padded_seqs) * 0.2)
# test_size = int(len(padded_seqs) * 0.1)

# train_ds = ds.take(train_size).batch(20)
# val_ds = ds.skip(train_size).take(val_size).batch(20)
# test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)

# # 하이퍼파라미터 설정
# dropout_prob = 0.5
# EMB_SIZE = 128
# EPOCH = 5
# VOCAB_SIZE = len(word_index) + 1  # 전체 단어 수

# # CNN 모델 정의
# input_layer = Input(shape=(MAX_SEQ_LEN,))
# embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
# dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

# conv1 = Conv1D(filters=128, kernel_size=3, padding='valid', activation=tf.nn.relu)(dropout_emb)
# pool1 = GlobalMaxPool1D()(conv1)
# conv2 = Conv1D(filters=128, kernel_size=4, padding='valid', activation=tf.nn.relu)(dropout_emb)
# pool2 = GlobalMaxPool1D()(conv2)
# conv3 = Conv1D(filters=128, kernel_size=5, padding='valid', activation=tf.nn.relu)(dropout_emb)
# pool3 = GlobalMaxPool1D()(conv3)

# # 3, 4, 5- gram 이후 합치기
# concat = concatenate([pool1, pool2, pool3])
# hidden = Dense(128, activation=tf.nn.relu)(concat)
# dropout_hidden = Dropout(rate=dropout_prob)(hidden)
# logits = Dense(3, name='logits')(dropout_hidden)
# predictions = Dense(3, activation=tf.nn.softmax)(logits)

# # 모델 생성
# model = Model(inputs=input_layer, outputs=predictions)
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # 모델 학습
# model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

# # 모델 평가(테스트 데이터셋 이용)
# loss, accuracy = model.evaluate(test_ds, verbose=1)
# print('Accuracy: %f' % (accuracy * 100))
# print('loss: %f' % (loss))

# # 모델 저장
# model.save('../MODL/cnn_model.h5')

corpus[0:10] [['12시', '땡'], ['1지망', '학교', '떨어졌어'], ['3박4일', '놀러가고', '싶다'], ['3박4일', '정도', '놀러가고', '싶다'], ['ppl', '심하네'], ['sd카드', '망가졌어'], ['sd카드', '안돼'], ['sns', '맞팔', '왜', '안하지ㅠㅠ'], ['sns', '시간낭비인', '거', '아는데', '매일', '하는', '중'], ['sns', '시간낭비인데', '자꾸', '보게됨']]
<class 'list'>
sequences[0:10] [[4646, 4647], [4648, 343, 448], [2580, 803, 11], [2580, 804, 803, 11], [4649, 2581], [2582, 4650], [2582, 64], [805, 4651, 14, 4652], [805, 4653, 3, 502, 238, 45, 106], [805, 4654, 23, 4655]]
<class 'numpy.ndarray'>
[[4646 4647    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [4648  343  448    0    0    0    0    0    0    0    0    0    0    0
     0]
 [2580  803   11    0    0    0    0    0    0    0    0    0    0    0
     0]
 [2580  804  803   11    0    0    0    0    0    0    0    0    0    0
     0]
 [4649 2581    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [2582 4650    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [2582   

In [11]:
# 6-4

import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing

# 데이터 읽어오기
train_file = "../../chatbot/book_ex/ch6/chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()

# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# 테스트용 데이터셋 생성
# ??? 위에서 정의한 test_ds 쓰지 않고 ds.take(2000)을 쓴 이유
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20) # 테스트 데이터셋

# 감정 분류 CNN 모델 불러오기
model = load_model('../MODL/cnn_model.h5')
model.summary()
model.evaluate(test_ds, verbose=2)   # loss & acc

# 테스트용 데이터셋의 10212번째 데이터 출력
print("단어 시퀀스 : ", corpus[10212])
print("단어 인덱스 시퀀스 : ", padded_seqs[10212])
print("문장 분류(정답) : ", labels[10212])

# 테스트용 데이터셋의 10212번째 데이터 감정 예측
picks = [10212]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print("감정 예측 점수 : ", predict)
print("감정 예측 클래스 : ", predict_class.numpy())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 15)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 15, 128)      1715072     ['input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 15, 128)      0           ['embedding[0][0]']              
                                                                                                  
 conv1d (Conv1D)                (None, 13, 128)      49280       ['dropout[0][0]']                
                                                                                              

In [15]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate


# 데이터 읽어오기
train_file = "../../chatbot/book_ex/ch6/chatbot_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Q'].tolist()
labels = data['label'].tolist()


# 단어 인덱스 시퀀스 벡터, 음절단위 분류
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
#print(corpus[0:10])


# 단어 토큰화??, 음절단위 분리 후 또하는 이유는?
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)


# 임베딩, 라벨 인코더
sequences = tokenizer.texts_to_sequences(corpus)  # 총 단어에 각기 다른 숫자를 부여한 리스트 반환 --> 임베딩의 한 방법중 라벨 인코딩 같음
word_index = tokenizer.word_index
print(type(sequences))
print(sequences[0:10])


# 15개 단어 차원으로 패딩이 들어감
MAX_SEQ_LEN = 15  # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')
print(type(padded_seqs))
print(padded_seqs[0:10])


# 학습용, 검증용, 테스트용 데이터셋 생성 ➌
# 학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)


def define_Model_Fit(ks_list, n_fileters, embed_vs, num_epoch, modl_name):
 

    # 하이퍼파라미터 설정
    dropout_prob = 0.5
    EMB_SIZE = embed_vs
    EPOCH = num_epoch
    VOCAB_SIZE = len(word_index) + 1  # 전체 단어 수

    # CNN 모델 정의
    input_layer = Input(shape=(MAX_SEQ_LEN,))
    embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
    dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

    conv1 = Conv1D(filters=n_fileters, kernel_size=ks_list[0], padding='valid', activation=tf.nn.relu)(dropout_emb)
    pool1 = GlobalMaxPool1D()(conv1)
    conv2 = Conv1D(filters=n_fileters, kernel_size=ks_list[1], padding='valid', activation=tf.nn.relu)(dropout_emb)
    pool2 = GlobalMaxPool1D()(conv2)
    conv3 = Conv1D(filters=n_fileters, kernel_size=ks_list[2], padding='valid', activation=tf.nn.relu)(dropout_emb)
    pool3 = GlobalMaxPool1D()(conv3)

    # 3, 4, 5- gram 이후 합치기
    concat = concatenate([pool1, pool2, pool3])
    hidden = Dense(128, activation=tf.nn.relu)(concat)
    dropout_hidden = Dropout(rate=dropout_prob)(hidden)
    logits = Dense(3, name='logits')(dropout_hidden)
    predictions = Dense(3, activation=tf.nn.softmax)(logits)

    # 모델 생성
    model = Model(inputs=input_layer, outputs=predictions)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # 모델 학습
    model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

    # 모델 평가(테스트 데이터셋 이용)
    loss, accuracy = model.evaluate(test_ds, verbose=1)
    print('Accuracy: %f' % (accuracy * 100))
    print('loss: %f' % (loss))

    # 모델 저장
    model.save('../MODL/'+modl_name)


if __name__ == '__main__':

    #exp1
    ks_list = [5,6,7]
    n_fileters =256
    embed_vs = 256
    num_epoch = 7
    define_Model_Fit(ks_list, n_fileters, embed_vs, num_epoch, "cnn_model_v2.h5")

    #exp2
    ks_list = [7,8,9]
    n_fileters = 512
    embed_vs = 512
    num_epoch = 9
    define_Model_Fit(ks_list, n_fileters, embed_vs, num_epoch, "cnn_model_v3.h5")

<class 'list'>
[[4646, 4647], [4648, 343, 448], [2580, 803, 11], [2580, 804, 803, 11], [4649, 2581], [2582, 4650], [2582, 64], [805, 4651, 14, 4652], [805, 4653, 3, 502, 238, 45, 106], [805, 4654, 23, 4655]]
<class 'numpy.ndarray'>
[[4646 4647    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [4648  343  448    0    0    0    0    0    0    0    0    0    0    0
     0]
 [2580  803   11    0    0    0    0    0    0    0    0    0    0    0
     0]
 [2580  804  803   11    0    0    0    0    0    0    0    0    0    0
     0]
 [4649 2581    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [2582 4650    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [2582   64    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [ 805 4651   14 4652    0    0    0    0    0    0    0    0    0    0
     0]
 [ 805 4653    3  502  238   45  106    0    0    0    0    0    0    0
     0]
 [ 805 4654   23 4655    0    0    0    0    0  

In [88]:
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))

#print(padded_seqs)

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
#test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size)


test_x = []
test_y = []
for i, (e1,e2) in enumerate(test_ds):
    #print(i, '-'*50)
    test_x.append(list(e1.numpy()))
    test_y.append(e2.numpy())
    #print(type(e1), e1.numpy())
    #print(type(e2), e2.numpy())
    #if i == 0: break

#print(x)

#print(test_ds.shape)
# 감정 분류 CNN 모델 불러오기
model1 = load_model('../MODL/cnn_model.h5')
model2 = load_model('../MODL/cnn_model_v2.h5')
model3 = load_model('../MODL/cnn_model_v3.h5')


# 다양하게 test_ds 전체 사용
#model1.evaluate(test_ds, verbose=2)   # loss & acc
#model2.evaluate(test_ds, verbose=2)   # loss & acc
#model3.evaluate(test_ds, verbose=2)   # loss & acc

pred_test_y1 = np.argmax(model1.predict(tf.convert_to_tensor(test_x)), axis=1)
pred_test_y2 = np.argmax(model2.predict(tf.convert_to_tensor(test_x)), axis=1)
pred_test_y3 = np.argmax(model3.predict(tf.convert_to_tensor(test_x)), axis=1)

print(classification_report(test_y, pred_test_y1))
print(classification_report(test_y, pred_test_y2))
print(classification_report(test_y, pred_test_y3))


37/37 [==============================] - 1s 13ms/step
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       529
           1       1.00      0.96      0.98       359
           2       0.99      0.98      0.98       294

    accuracy                           0.98      1182
   macro avg       0.98      0.98      0.98      1182
weighted avg       0.98      0.98      0.98      1182

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       529
           1       0.99      0.98      0.99       359
           2       0.99      1.00      0.99       294

    accuracy                           0.99      1182
   macro avg       0.99      0.99      0.99      1182
weighted avg       0.99      0.99      0.99      1182

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       529
           1       1.00      0.98      0.99       359
           2       1.